In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
import json
import pickle

import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Activation, Dropout
from keras.optimizers import SGD
import random

[nltk_data] Downloading package punkt to /Users/rajarajen/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/rajarajen/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
Using TensorFlow backend.


In [12]:
## Initialize Training
words=[]
classes = []
documents = []
ignore_words = ['?', '!']
data_file = open('../data/training/intents2.json').read()
intents = json.loads(data_file)

In [14]:
for intent in intents['intents']:
    for pattern in intent['patterns']:

        # take each word and tokenize it
        w = nltk.word_tokenize(pattern)
        words.extend(w)
        # adding documents
        documents.append((w, intent['tag']))

        # adding classes to our class list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [15]:
words = [lemmatizer.lemmatize(w.lower()) for w in words if w not in ignore_words]
words = sorted(list(set(words)))

classes = sorted(list(set(classes)))

print (len(documents), "documents")

print (len(classes), "classes", classes)

print (len(words), "unique lemmatized words", words)


pickle.dump(words,open('../data/training/created/words2.pkl','wb'))
pickle.dump(classes,open('../data/training/created/classes2.pkl','wb'))

94 documents
9 classes ['adverse_drug', 'blood_pressure', 'blood_pressure_search', 'goodbye', 'greeting', 'hospital_search', 'options', 'pharmacy_search', 'thanks']
88 unique lemmatized words ["'s", ',', 'a', 'adverse', 'all', 'anyone', 'are', 'awesome', 'be', 'behavior', 'blood', 'by', 'bye', 'can', 'causing', 'chatting', 'check', 'could', 'data', 'day', 'detail', 'do', 'dont', 'drug', 'entry', 'find', 'for', 'give', 'good', 'goodbye', 'have', 'hello', 'help', 'helpful', 'helping', 'hey', 'hi', 'history', 'hola', 'hospital', 'how', 'i', 'id', 'is', 'later', 'list', 'load', 'locate', 'log', 'looking', 'lookup', 'management', 'me', 'module', 'nearby', 'next', 'nice', 'of', 'offered', 'open', 'patient', 'pharmacy', 'pressure', 'provide', 'reaction', 'related', 'result', 'search', 'searching', 'see', 'show', 'suitable', 'support', 'task', 'thank', 'thanks', 'that', 'there', 'till', 'time', 'to', 'transfer', 'up', 'want', 'what', 'which', 'with', 'you']


In [16]:
# initializing training data
training = []
output_empty = [0] * len(classes)
for doc in documents:
    # initializing bag of words
    bag = []
    # list of tokenized words for the pattern
    pattern_words = doc[0]
    # lemmatize each word - create base word, in attempt to represent related words
    pattern_words = [lemmatizer.lemmatize(word.lower()) for word in pattern_words]
    # create our bag of words array with 1, if word match found in current pattern
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)

    # output is a '0' for each tag and '1' for current tag (for each pattern)
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1

    training.append([bag, output_row])
# shuffle our features and turn into np.array
random.shuffle(training)
training = np.array(training)
# create train and test lists. X - patterns, Y - intents
train_x = list(training[:,0])
train_y = list(training[:,1])
print("Training data created")

Training data created


In [17]:
# Create model - 3 layers. First layer 128 neurons, second layer 64 neurons and 3rd output layer contains number of neurons
# equal to number of intents to predict output intent with softmax
model = Sequential()
model.add(Dense(128, input_shape=(len(train_x[0]),), activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.5))
model.add(Dense(len(train_y[0]), activation='softmax'))

# Compile model. Stochastic gradient descent with Nesterov accelerated gradient gives good results for this model
sgd = SGD(lr=0.01, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy', optimizer=sgd, metrics=['accuracy'])

#fitting and saving the model
hist = model.fit(np.array(train_x), np.array(train_y), epochs=200, batch_size=5, verbose=1)
model.save('chatbot_model.h5', hist)

print("model created")

Epoch 1/200
94/94 [==============================] - 0s 1ms/step - loss: 2.2104 - accuracy: 0.1064
Epoch 2/200
94/94 [==============================] - 0s 191us/step - loss: 2.0308 - accuracy: 0.3617
Epoch 3/200
94/94 [==============================] - 0s 191us/step - loss: 1.8485 - accuracy: 0.4362
Epoch 4/200
94/94 [==============================] - 0s 188us/step - loss: 1.6377 - accuracy: 0.4787
Epoch 5/200
94/94 [==============================] - 0s 191us/step - loss: 1.3339 - accuracy: 0.6596
Epoch 6/200
94/94 [==============================] - 0s 169us/step - loss: 1.2018 - accuracy: 0.6277
Epoch 7/200
94/94 [==============================] - 0s 168us/step - loss: 1.0592 - accuracy: 0.7021
Epoch 8/200
94/94 [==============================] - 0s 162us/step - loss: 0.8655 - accuracy: 0.7021
Epoch 9/200
94/94 [==============================] - 0s 157us/step - loss: 0.7562 - accuracy: 0.7872
Epoch 10/200
94/94 [==============================] - 0s 168us/step - loss: 0.6597 - accuracy

94/94 [==============================] - 0s 156us/step - loss: 0.0150 - accuracy: 1.0000
Epoch 82/200
94/94 [==============================] - 0s 164us/step - loss: 0.0194 - accuracy: 1.0000
Epoch 83/200
94/94 [==============================] - 0s 164us/step - loss: 0.0118 - accuracy: 1.0000
Epoch 84/200
94/94 [==============================] - 0s 160us/step - loss: 0.0104 - accuracy: 1.0000
Epoch 85/200
94/94 [==============================] - 0s 170us/step - loss: 0.0190 - accuracy: 1.0000
Epoch 86/200
94/94 [==============================] - 0s 164us/step - loss: 0.0475 - accuracy: 0.9894
Epoch 87/200
94/94 [==============================] - 0s 166us/step - loss: 0.0079 - accuracy: 1.0000
Epoch 88/200
94/94 [==============================] - 0s 164us/step - loss: 0.0167 - accuracy: 1.0000
Epoch 89/200
94/94 [==============================] - 0s 163us/step - loss: 0.0501 - accuracy: 0.9894
Epoch 90/200
94/94 [==============================] - 0s 167us/step - loss: 0.0230 - accuracy: 

94/94 [==============================] - 0s 169us/step - loss: 0.0283 - accuracy: 0.9787
Epoch 161/200
94/94 [==============================] - 0s 166us/step - loss: 0.0092 - accuracy: 1.0000
Epoch 162/200
94/94 [==============================] - 0s 160us/step - loss: 0.0130 - accuracy: 1.0000
Epoch 163/200
94/94 [==============================] - 0s 155us/step - loss: 0.0102 - accuracy: 1.0000
Epoch 164/200
94/94 [==============================] - 0s 156us/step - loss: 0.0136 - accuracy: 1.0000
Epoch 165/200
94/94 [==============================] - 0s 168us/step - loss: 0.0035 - accuracy: 1.0000
Epoch 166/200
94/94 [==============================] - 0s 160us/step - loss: 0.0139 - accuracy: 1.0000
Epoch 167/200
94/94 [==============================] - 0s 165us/step - loss: 0.0155 - accuracy: 0.9894
Epoch 168/200
94/94 [==============================] - 0s 168us/step - loss: 0.0047 - accuracy: 1.0000
Epoch 169/200
94/94 [==============================] - 0s 152us/step - loss: 0.0043 - a